In [1]:
pip install keras-buoy

  Created wheel for keras-buoy: filename=keras_buoy-0.1.4.1-py2.py3-none-any.whl size=6703 sha256=2e805ddd73e502cbd65dee177e319e35631395c307353ad1a91aedcd1a5b8a44
  Stored in directory: /root/.cache/pip/wheels/4c/8c/65/cb5c62db3a0e046a72cd2c5b6493c48c58b0029754488d2830
Successfully built keras-buoy


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [4]:
# Inicializar la CNN
classifier = Sequential()

# paso 1 convolucion
classifier.add(Conv2D( filters = 32, kernel_size = (3, 3), input_shape = (64, 64, 3), activation="relu"))

# paso 2 max pooling
classifier.add(MaxPooling2D(pool_size=(2,2))) # 2x2 is the optimal value

# segunda capa de convolucion y max pooling
classifier.add(Conv2D( filters = 32, kernel_size = (3, 3), activation="relu"))
classifier.add(MaxPooling2D(pool_size=(2,2)))

# tercera capa de convolucion y max pooling pero con mas filtros para aumentar la busqueda de filtros
classifier.add(Conv2D( filters = 64, kernel_size = (3, 3), activation="relu"))
classifier.add(MaxPooling2D(pool_size=(2,2)))

# paso 3 flattering
classifier.add(Flatten())

In [5]:
# Full conection
classifier.add(Dense(units = 128, activation = "relu"))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 1, activation = "sigmoid"))



# Compilar la red neuronal
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *

In [7]:
filepath="/content/drive/MyDrive/deep learning/deep_learning_a-z/Part 2 - Convolutional Neural Networks (CNN)/epochs/"
#checkpoint_{epoch:02d}_{batch:04d}.hdf5
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='auto', save_weights_only=True)
callbacks_list = [checkpoint]

In [8]:
from keras_buoy.models import ResumableModel

In [ ]:
# ajustar la red a las imagenes
path = '/content/drive/MyDrive/deep learning/deep_learning_a-z/Part 2 - Convolutional Neural Networks (CNN)/dataset/'
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory(
        path + 'training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

testing_dataset = test_datagen.flow_from_directory(
        path + 'test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

checkpoints_path='/content/drive/MyDrive/deep learning/deep_learning_a-z/Part 2 - Convolutional Neural Networks (CNN)/checkpoints/model_weights.h5'
 
resumable_model = ResumableModel(classifier, 
                                 save_every_epochs=1, 
                                 custom_objects=None, 
                                 to_path=checkpoints_path)

history = resumable_model.fit(x=training_dataset, 
                              validation_data=testing_dataset, 
                              batch_size=32, 
                              steps_per_epoch = 8000/32,
                              epochs=90, 
                              validation_steps = 2000)


Found 8011 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 14/90
 80/250 [========>.....................] - ETA: 16:58 - loss: 0.6932 - accuracy: 0.5082

In [ ]:
classifier.save('/content/drive/MyDrive/deep learning/deep_learning_a-z/Part 2 - Convolutional Neural Networks (CNN)/model_trained.h5')

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
# predicion 1
test_image = image.load_img(path + 'single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = classifier.predict(test_image)
print(result[0][0])

# predicion 2
test_image = image.load_img(path + 'single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result2 = classifier.predict(test_image)
print(result2[0][0])

# otros tests
test_image = image.load_img('/content/drive/MyDrive/deep learning/deep_learning_a-z/Part 2 - Convolutional Neural Networks (CNN)/dataset/test_set/cats/cat.4001.jpg', target_size = (64, 64))
plt.imshow(test_image)
plt.show()
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
 
result3 = classifier.predict(test_image)
print(result3[0][0])

if result[0][0] < 1.0:
  print('cat')
else:
  print('dog')

